In [1]:
import os
import math as math
import numpy as np

# Utilities
import Utils.folder_utils as folder_utils
import Utils.TIMIT_utils as TIMIT_utils
import Utils.image_utils as image_utils

# Sound utils
from Utils.nist_reader import NistReader
import Utils.sound_utils as sound_utils

# Spectrograms
from Utils.SpectrogramFactory import SpectrogramFactory
from Utils.Spectrogram import Spectrogram

import Utils.Spectrogram as SpectrogramUtils

# Image utils
from PIL import Image

%matplotlib inline

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
#sound
path_to_flac = "Data/TIMIT/timit/train"
path_to_flac_test_folder = "TestFolder/TestData/fcjf0"
path_to_flac_test_folder_track = "TestFolder/TestData/fcjf0/SA1.WAV"

#temp
temp_folder = "TestFolder/Temp"
temp_phonemes_folder = "TestFolder/TempPhonemes"
test_output = "TestFolder/Temp/results.wav"
#temp_image_to_process = "/Volumes/Storage/processing.jpg"

In [3]:
pathes = folder_utils.reverse_folder(path_to_flac_test_folder, ".WAV")

In [4]:
pathes

['TestFolder/TestData/fcjf0/SA1.WAV',
 'TestFolder/TestData/fcjf0/SA2.WAV',
 'TestFolder/TestData/fcjf0/SI1027.WAV',
 'TestFolder/TestData/fcjf0/SI1657.WAV',
 'TestFolder/TestData/fcjf0/SI648.WAV',
 'TestFolder/TestData/fcjf0/SX127.WAV',
 'TestFolder/TestData/fcjf0/SX217.WAV',
 'TestFolder/TestData/fcjf0/SX307.WAV',
 'TestFolder/TestData/fcjf0/SX37.WAV',
 'TestFolder/TestData/fcjf0/SX397.WAV']

In [5]:
nistReader = NistReader()
spectrogramFactory = SpectrogramFactory()
phonemeOffset = 64 * 5

In [6]:
path = pathes[0]
#sentence
print path
phonemes = TIMIT_utils.parse_phoneme_file(path)
speaker = folder_utils.get_speaker_name(path)

temp_speaker_folder = os.path.join(temp_folder, speaker)
if not os.path.exists(temp_speaker_folder):
    os.makedirs(temp_speaker_folder)
    
#converting to wave
wav_file = nistReader.Nist2Wav(path, temp_speaker_folder)
#wav_file = SU.PCM2Wav(path, temp_speaker_folder)

for i in range(len(phonemes)):
    phoneme = phonemes[i]
    #Cutting one phoneme
    if i == 0 or i == len(phonemes):
        start = int(phoneme[0])
        end = int(phoneme[1])
    else:
        start = int(phoneme[0]) - phonemeOffset
        end = int(phoneme[1]) + phonemeOffset
        
    # TODO: create a phoneme object
    phone_file = sound_utils.cutPhonemeChunk(wav_file, temp_phonemes_folder, start, end, phoneme[2])
    spectrogram = spectrogramFactory.create_spectrogram(phone_file) 

TestFolder/TestData/fcjf0/SA1.WAV


In [ ]:
def InvertOneSpectrogram(spectrogram, window_size, frame_step):
    _fftSize, _frameCount = spectrogram.shape
    windowSize = math.floor(_fftSize/2);
    waveSize = (_frameCount - 1) * frame_step + window_size
    waveform = np.zeros(waveSize);
    # To sum up total windowing effect
    totalWindowingSum = np.zeros(waveSize);
    h = 0.54 - 0.46*np.cos(2*math.pi* np.arange(window_size)/(window_size-1))
    fftB = int(math.floor(window_size/2))
    fftE = int(fftB + window_size)
    for frameNumber in range(0,_frameCount):
        waveB = (frameNumber)*frame_step
        waveE = waveB + window_size
        spectralSlice = spectrogram[:, frameNumber]
        newFrame = np.fft.ifft(spectralSlice);
        newFrame = np.real(np.fft.ifft(spectralSlice));
        waveform[waveB:waveE] = waveform[waveB:waveE] + newFrame[fftB:fftE]
        totalWindowingSum[waveB:waveE] = totalWindowingSum[waveB:waveE] + h;
    waveform = np.divide(np.real(waveform),totalWindowingSum);
    return waveform

In [12]:
window_size = 256
frame_step = 64

spectrogramObj = spectrogramFactory.create_spectrogram(wav_file) 
spectrogram = spectrogramObj.spectrogram_values

waveform = SpectrogramUtils.InvertSpectrogram(spectrogram)
sound_utils.save_to_wav(test_output, waveform)